In [ ]:
import os
arch = os.getenv("ARGS", "real")

In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import ufl
    import dolfinx
else:
    try:
        import ufl
        import dolfinx
    except ImportError:
        if arch != "complex":
            !wget "https://fem-on-colab.github.io/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
        else:
            !wget "https://fem-on-colab.github.io/releases/fenicsx-install-complex.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
        import ufl
        import dolfinx

In [ ]:
import numpy as np
import petsc4py
import petsc4py.PETSc
if arch != "complex":
    assert not np.issubdtype(petsc4py.PETSc.ScalarType, np.complexfloating)
else:
    assert np.issubdtype(petsc4py.PETSc.ScalarType, np.complexfloating)

In [ ]:
import dolfinx.fem
import dolfinx.mesh

In [ ]:
import mpi4py
import mpi4py.MPI

In [ ]:
mesh = dolfinx.mesh.create_unit_interval(mpi4py.MPI.COMM_WORLD, 3)

In [ ]:
V = dolfinx.fem.FunctionSpace(mesh, ("CG", 1))

In [ ]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

In [ ]:
dx = ufl.dx

In [ ]:
f = dolfinx.fem.Function(V)
dim_V = f.vector.local_size
assert dim_V == 4
f.vector[:] = np.arange(1, dim_V + 1)

In [ ]:
a = ufl.inner(u, v) * dx
F = ufl.inner(f, v) * dx
a_cpp = dolfinx.fem.form(a)
F_cpp = dolfinx.fem.form(F)

In [ ]:
A = dolfinx.fem.petsc.assemble_matrix(a_cpp)
b = dolfinx.fem.petsc.assemble_vector(F_cpp)

In [ ]:
A.assemble()

In [ ]:
solution = dolfinx.fem.Function(V)

In [ ]:
for package in (None, "mumps", "superlu", "superlu_dist"):
    ksp = petsc4py.PETSc.KSP().create()
    ksp.setOperators(A)
    ksp.setType("preonly")
    ksp.getPC().setType("lu")
    if package is not None:
        ksp.getPC().setFactorSolverType(package)
    ksp.setFromOptions()
    ksp.solve(b, solution.vector)
    solution.vector.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
    assert np.allclose(solution.vector.getArray(), np.arange(1, dim_V + 1))

In [ ]:
k = ufl.inner(ufl.grad(u), ufl.grad(v)) * dx
k_cpp = dolfinx.fem.form(k)

In [ ]:
K = dolfinx.fem.petsc.assemble_matrix(k_cpp)
K.assemble()

In [ ]:
expected = (0, 10.8, 54, 108)

In [ ]:
import slepc4py
import slepc4py.SLEPc

In [ ]:
for package in (None, "mumps", "superlu", "superlu_dist"):
    eps = slepc4py.SLEPc.EPS().create()
    eps.setOperators(K, A)
    eps.setProblemType(slepc4py.SLEPc.EPS.ProblemType.GHEP)
    eps.setWhichEigenpairs(slepc4py.SLEPc.EPS.Which.TARGET_REAL)
    eps.setTarget(1)
    st = eps.getST()
    st.setType(slepc4py.SLEPc.ST.Type.SINVERT)
    st.setShift(1)
    ksp = st.getKSP()
    ksp.setType("preonly")
    pc = ksp.getPC()
    pc.setType("lu")
    if package is not None:
        pc.setFactorSolverType(package)
    eps.solve()
    assert eps.getConverged() == len(expected)
    for (i, eig_i_ex) in enumerate(expected):
        eig_i = eps.getEigenvalue(i)
        assert np.isclose(eig_i.real, eig_i_ex)
        assert np.isclose(eig_i.imag, 0)

In [ ]:
%%bash

export LD_PRELOAD=""
ERROR_LIBRARIES=($(find /root/.cache/fenics -name '*\.so' -exec \
    bash -c 'ldd $0 | grep libstdc++.so.6 1>/dev/null 2>/dev/null && echo $0' {} \;))
if [ ${#ERROR_LIBRARIES[@]} -eq 0 ]; then
    echo "No reference to libstdc++.so was found"
else
    for ERROR_LIBRARY in "${ERROR_LIBRARIES[@]}"; do
        echo "Error: library $ERROR_LIBRARY depends on libstdc++.so"
        ldd -v $ERROR_LIBRARY
    done
    false
fi